In [3]:
    import ipywidgets, jinja2.meta, IPython, traitlets, nbformat, annodize, textwrap

In [4]:
    class Template(ipywidgets.HTML):
        unnamed_references = traitlets.Set()
        environment = traitlets.Instance(jinja2.Environment)
        data = traitlets.Dict()
        template = traitlets.Instance(jinja2.Template)
        
        def __init__(self, body, **data):
            super().__init__(template=jinja2.Template(body), environment=jinja2.Environment())
            self.unnamed_references = jinja2.meta.find_undeclared_variables(self.environment.parse(body))
            self.data = data
            self.add_traits(**{
                k: traitlets.Any() for k in self.unnamed_references
            })
            self.observe(self._update_key, list(self.unnamed_references))
                
        def _update_key(self, change): 
            self.data[change['name']] = change['new']
            self._update_data({'new': self.data})
            
        @traitlets.observe('data')
        def _update_data(self, change):
            self.value = self.template.render({k: v for k, v in change['new'].items() if k in self.unnamed_references})